In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import layers

In [97]:
class Inception_Module(tf.keras.layers.Layer):
    def __init__(self, filters):
        super(Inception_Module, self).__init__()
        self.conv_1x1 = layers.Conv2D(filters[0], 1, 1, padding = 'same', activation = 'relu')
        
        self.conv_3x3_reduce = layers.Conv2D(filters[1], 3, 1, padding = 'same', activation = 'relu')
        self.conv_3x3 = layers.Conv2D(filters[2], 3, 1, padding = 'same', activation = 'relu')
        
        self.conv_5x5_reduce = layers.Conv2D(filters[3], 5, 1, padding = 'same', activation = 'relu')
        self.conv_5x5 = layers.Conv2D(filters[4], 5, 1, padding = 'same', activation = 'relu')
        
        self.pooling = layers.MaxPooling2D(pool_size=(3, 3), strides = (1, 1), padding = 'same')
        self.pool_1x1 = layers.Conv2D(filters[5], 1, 1, padding = 'same', activation = 'relu')
        
    def call(self, inputs):
        out1 = self.conv_1x1(inputs)
        out2 = self.conv_3x3_reduce(inputs)
        out3 = self.conv_5x5_reduce(inputs)
        out4 = self.pooling(inputs)
        
        out_2_2 = self.conv_3x3(out2)
        out_3_2 = self.conv_5x5(out3)
        out_4_2 = self.pool_1x1(out4)
        
        output = tf.concat([out1, out_2_2, out_3_2, out_4_2], axis = -1)
        
        return output

In [235]:
class Inception(tf.keras.Model):
    def __init__(self):
        super(Inception, self).__init__()
        self.input_layer = layers.Conv2D(64, 7, 2, activation = 'relu', padding = 'same', input_shape = (224, 224, 3))
        self.pool1 = layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')
        self.bn1 = layers.BatchNormalization()
        
        self.conv1 = layers.Conv2D(192, 3, 1, padding = 'same', activation = 'relu')
        self.pool2 = layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')
        self.bn2 = layers.BatchNormalization()
        
        self.inception_3a = Inception_Module([64, 96, 128, 16, 32, 32])
        self.inception_3b = Inception_Module([128, 128, 192, 32, 96, 64])
        
        self.pool3 = layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')
        
        self.inception_4a = Inception_Module([192, 96, 208, 16, 48, 64]) # output 1
        self.out_avg_1 = layers.AveragePooling2D((5, 5), strides = 3)
        self.out_1x1_1 = layers.Conv2D(128, 1, 1, activation='relu')
        self.out_fc1_1 = layers.Dense(1024, activation = 'relu')
        self.dropout_1 = layers.Dropout(0.7)
        self.out_fc2_1 = layers.Dense(10)
        
        self.inception_4b = Inception_Module([160, 112, 224, 24, 64, 64])
        self.inception_4c = Inception_Module([128, 128, 256, 24, 64, 64]) 
        self.inception_4d = Inception_Module([112, 144, 288, 32, 64, 64]) # output 2
        self.out_avg_2 = layers.AveragePooling2D((5, 5), strides = 3)
        self.out_1x1_2 = layers.Conv2D(128, 1, 1, activation='relu')
        self.out_fc1_2 = layers.Dense(1024, activation = 'relu')
        self.dropout_2 = layers.Dropout(0.7)
        self.out_fc2_2 = layers.Dense(10)
        
        self.flatten = layers.Flatten()
        
        
        self.inception_4e = Inception_Module([256, 160, 320, 32, 128, 128])
        
        self.pool4 = layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')
        
        self.inception_5a = Inception_Module([256, 160, 320, 32, 128, 128])
        self.inception_5b = Inception_Module([384, 192, 384, 48, 128, 128])
        
        self.avg_pool = layers.AveragePooling2D((7, 7), strides = 1)
        self.dropout = layers.Dropout(0.7)
        self.fc = layers.Dense(10)
        
    def call(self, inputs):
        out = self.input_layer(inputs)
        
        out = self.pool1(out)
        out = self.bn1(out)
        
        out = self.conv1(out)
        out = self.pool2(out)
        out = self.bn2(out)
        
        out = self.inception_3a(out)
        out = self.inception_3b(out)
        
        out = self.pool3(out)
        
        out = self.inception_4a(out) # Auxiliary output 1
        output1 = self.out_avg_1(out)
        output1 = self.out_1x1_1(output1)
        output1 = self.flatten(output1)
        output1 = self.out_fc1_1(output1)
        output1 = self.dropout_1(output1)
        output1 = self.out_fc2_1(output1)
        
        out = self.inception_4b(out)
        out = self.inception_4c(out)
        
        out = self.inception_4d(out) # Auxiliary output 2
        output2 = self.out_avg_2(out)
        output2 = self.out_1x1_2(output2)
        output2 = self.flatten(output2)
        output2 = self.out_fc1_2(output2)
        output2 = self.dropout_2(output2)
        output2 = self.out_fc2_2(output2)
        
        out = self.inception_4e(out)
        
        out = self.pool4(out)
        
        out = self.inception_5a(out)
        out = self.inception_5b(out)
        
        out = self.avg_pool(out)
        out = self.dropout(out)
        output3 = self.fc(out)
        
        return [output1, output2, tf.reshape(output3, (-1, 10))]

In [236]:
model = Inception()

In [237]:
sample = tf.constant([10*224*224*3], dtype=tf.float32, shape = (10, 224, 224, 3))
sample_label = tf.cast(list(range(10)), dtype=tf.float32)

In [238]:
Inception_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=False),
                 loss = [Inception_loss, Inception_loss, Inception_loss], loss_weights = [1, 0.3, 0.3],
                 metrics = ['acc'])

In [239]:
model.fit(sample, [sample_label, sample_label, sample_label], epochs = 3, batch_size = 4, verbose = 1)

Epoch 1/3
3/3 [==============================] - 0s 28ms/step - loss: 3.7222 - output_1_loss: 2.3511 - output_2_loss: 2.3767 - output_3_loss: 2.1939 - output_1_acc: 0.2000 - output_2_acc: 0.2000 - output_3_acc: 0.3000
Epoch 2/3
3/3 [==============================] - 0s 28ms/step - loss: 3.8189 - output_1_loss: 2.2808 - output_2_loss: 2.5030 - output_3_loss: 2.6241 - output_1_acc: 0.1000 - output_2_acc: 0.1000 - output_3_acc: 0.0000e+00
Epoch 3/3
3/3 [==============================] - 0s 28ms/step - loss: 3.8032 - output_1_loss: 2.3778 - output_2_loss: 2.2350 - output_3_loss: 2.5163 - output_1_acc: 0.2000 - output_2_acc: 0.1000 - output_3_acc: 0.0000e+00
